<a href="https://colab.research.google.com/github/brownt47/Python_Projects/blob/main/Course_Enrollment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import packages and libraries

In [ ]:
#pip install xlsxwriter
#import xlsxwriter

In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from pytz import timezone

# PRE_PROCESSING

### IMPORT THE COURSE SECTION DATA

In [ ]:
filename = 'CV_Class_Schedule_By_Department_S200106-2020-07-16_20210826_091257.csv'

In [ ]:
#loc = "C:\\Users\\User\\OneDrive - Georgia State University\\!!!Online Math Department\\Schedules for Online\\Fall 2021"
#loc = loc + "\\" + filename

In [ ]:
stats = pd.read_csv(filename)

## Change Column Names and remove unwanted columns

In [ ]:
stats.columns = [x.lower() for x in stats.columns]
stats.columns = ['term', 'coll', 'crn', 'dept', 'sapr', 'xlst', 'ptrm', 'subj_crsnumb',
       'crse_title', 'seq_numb', 'schd', 'insm_code', 'crse_delvry', 'campus',
       'sess', 'credit_hr_low', 'credit_hr_high', 'credit_hrs', 'sest', 'ses#',
       'days', 'start_time', 'end_time', 'schd.1', 'loc', 'instructor',
       'spriden_id', 'fcst_code', 'enroll', 'max_enrl', 'xlst_enrl',
       'xlst_max_enrl', 'crse_status', 'comments', 'fees', 'detail code',
       'sec', 'ssbsect_voice_avail']

## Label Support Courses

In [ ]:
support_list=(stats.comments.str.find('CRN') != -1)
insert_support_list = []
for x in support_list:
  if x == True:
   insert_support_list.append('Support')
  else: 
    insert_support_list.append('Clean')
stats = stats.assign(support=insert_support_list)

## Label Subjects for courses (Math, Eng, CS)

In [ ]:
subject_list=stats.subj_crsnumb
insert_subject_list = []
for x in subject_list:
  insert_subject_list.append(str(x).split(" ")[0])
stats = stats.assign(subject=insert_subject_list)

## Label PLC status for courses

In [ ]:
PLC_list=stats.comments.str.find('Perimeter Learning Community') != -1
insert_PLC_list = []
for x in PLC_list:
  if x == True:
   insert_PLC_list.append('PLC')
  else: 
    insert_PLC_list.append('Non-PLC')
stats = stats.assign(PLC = insert_PLC_list)

In [ ]:
data_columns = ['crn', 'dept','subject','xlst','subj_crsnumb',
       'crse_title', 'seq_numb', 'campus',
       'days', 'start_time', 'end_time', 'loc', 'instructor',
       'spriden_id', 'fcst_code', 'enroll', 'max_enrl', 'xlst_enrl',
       'xlst_max_enrl','PLC','support' ,'comments']

In [ ]:
enrollment =stats[data_columns]


### Create "seats_left" column

In [ ]:
enrollment = enrollment.assign(seats_left=enrollment['max_enrl'] - enrollment['enroll'])

In [ ]:
enrollment.columns

Index(['crn', 'dept', 'subject', 'xlst', 'subj_crsnumb', 'crse_title',
       'seq_numb', 'campus', 'days', 'start_time', 'end_time', 'loc',
       'instructor', 'spriden_id', 'fcst_code', 'enroll', 'max_enrl',
       'xlst_enrl', 'xlst_max_enrl', 'PLC', 'support', 'comments',
       'seats_left'],
      dtype='object')

### List of campuses

In [ ]:
campuses = ['PA','PF','PS','PC','PE', 'PN']

# Summary Open Seats Count

In [ ]:
filter = (enrollment['campus']=='PC') & (enrollment['subject']=='MATH')
#filter = (enrollment['subject']=='MATH')
#course_sums = enrollment[filter].groupby(['subject','campus','subj_crsnumb']).sum()
course_sums = enrollment[filter].groupby(['PLC','subject','campus','support','subj_crsnumb']).sum()
print(course_sums.columns)
course_sums['seats_left']

Index(['crn', 'dept', 'seq_numb', 'end_time', 'spriden_id', 'enroll',
       'max_enrl', 'xlst_enrl', 'xlst_max_enrl', 'seats_left'],
      dtype='object')


PLC      subject  campus  support  subj_crsnumb
Non-PLC  MATH     PC      Clean    MATH 1001       63
                                   MATH 1111       35
                                   MATH 1113       46
                                   MATH 1401       81
                                   MATH 2008       23
                                   MATH 2211       51
                                   MATH 2212       16
                                   MATH 2551       15
                                   MATH 2652       19
                          Support  MATH 0996        0
                                   MATH 0997       12
                                   MATH 0999        1
                                   MATH 1001       12
                                   MATH 1111       13
                                   MATH 1401        0
PLC      MATH     PC      Clean    MATH 1001        0
                                   MATH 1401        3
                          Support 

# Create Excel export object

#### Create filename for report with timestamp

In [ ]:
eastern = timezone('US/Eastern')
dateTimeObj = datetime.now(eastern)
datestamp = dateTimeObj.strftime("%Y%b%d_%I_%M%p")
report_name = "Empty_Seat_Report_" + datestamp + ".xlsx"
print(report_name)

Empty_Seat_Report_2021Aug26_09_13AM.xlsx


In [ ]:
writer = pd.ExcelWriter(report_name)

In [ ]:
# book object to format
workbook = writer.book  

## Create Sheets for each campus

In [ ]:
for campus in campuses:
  filter = (enrollment['campus']==campus) & (enrollment['subject']=='MATH')
  course_sums = enrollment[filter].groupby(['PLC','subject','campus','support','subj_crsnumb']).sum()
  course_sums['seats_left'].to_excel(writer, campus)
  worksheet = writer.sheets[campus]
'''  worksheet.set_column('A:D', 12)
  worksheet.set_column('E:E', 20)   #works with xlsrWriter package
  worksheet.set_column('F:F', 12)'''

"  worksheet.set_column('A:D', 12)\n  worksheet.set_column('E:E', 20)   #works with xlsrWriter package\n  worksheet.set_column('F:F', 12)"

## Export to Excel file

In [ ]:
# save the excel
writer.save()

# Summary Open Seats Count

In [ ]:
'''#campus_filter = enrollment['campus']=='PF'
#campus_filter = True   #for all campuses
#campus_filter
#course_filter = enrollment['subj_crsnumb']=='0996'
#course_filter = True   
# for all courses
#filter = campus_filter & course_filter
filter = True
Instructor_lists = enrollment.sort_values('subj_crsnumb')
#Instructor_lists = enrollment[filter].sort_values('subj_crsnumb')
Instructor_lists[['subj_crsnumb','instructor']].drop_duplicates()'''

"#campus_filter = enrollment['campus']=='PF'\n#campus_filter = True   #for all campuses\n#campus_filter\n#course_filter = enrollment['subj_crsnumb']=='0996'\n#course_filter = True   \n# for all courses\n#filter = campus_filter & course_filter\nfilter = True\nInstructor_lists = enrollment.sort_values('subj_crsnumb')\n#Instructor_lists = enrollment[filter].sort_values('subj_crsnumb')\nInstructor_lists[['subj_crsnumb','instructor']].drop_duplicates()"

In [ ]:
'''
#filter = (enrollment['campus']=='PA') & (enrollment['subject']=='MATH')
#low_enrollment_crns = enrollment[filter]
#xlist_low_enrollment'''

"\n#filter = (enrollment['campus']=='PA') & (enrollment['subject']=='MATH')\n#low_enrollment_crns = enrollment[filter]\n#xlist_low_enrollment"